In [63]:
import os
import requests
import csv
import re
import numpy as np

In [37]:
pdb_ext = []
with open('PHAS0052 PDB File - Sheet1.csv') as csvDataFile:
    read = csv.reader(csvDataFile)
    for i in read:
        pdb_ext.append(i[0])

pdb_info = [(entry[:-1].lower(), entry[-1]) for entry in pdb_ext]

print(pdb_info)

[('6ksh', 'D'), ('4cta', 'A'), ('2x14', 'A'), ('2j9c', 'A'), ('3f5m', 'A'), ('3ruv', 'D'), ('5trd', 'A'), ('3c1m', 'C'), ('3v2u', 'C'), ('1f3f', 'C'), ('7nsd', 'A'), ('5dd7', 'A'), ('1i58', 'A'), ('3hy2', 'Y'), ('7uld', 'A'), ('1z0s', 'A'), ('1un9', 'A'), ('7alr', 'A'), ('121p', 'A'), ('2jg1', 'C'), ('6cau', 'A'), ('2py7', 'X'), ('5dgh', 'A'), ('2aqx', 'B'), ('3dnt', 'A'), ('3wgu', 'C'), ('4uxx', 'C'), ('2xan', 'A'), ('6ci7', 'C'), ('3gqk', 'A'), ('4amf', 'A'), ('1d4x', 'A'), ('7cqq', 'A'), ('4lac', 'C'), ('7d8i', 'A'), ('1xdp', 'A'), ('2i1o', 'A'), ('1vl1', 'A'), ('1to6', 'A'), ('1yzy', 'A'), ('1fit', 'A'), ('1s1d', 'A'), ('6a8p', 'B'), ('2bz0', 'A'), ('3zcb', 'A'), ('4edk', 'A'), ('5guf', 'A'), ('1k90', 'A'), ('4crj', 'A'), ('6aaz', 'A'), ('6fl4', 'A'), ('8dcd', 'A'), ('1twf', 'B'), ('7tgk', 'D'), ('1rn8', 'A'), ('1j09', 'A'), ('2q16', 'B'), ('7edz', 'C'), ('1wc6', 'B'), ('3tux', 'A'), ('3vth', 'A'), ('6p1p', 'A'), ('6r5d', 'A'), ('4yvz', 'A'), ('4ru9', 'A'), ('3f2b', 'A'), ('3erc', 

In [9]:
for pdb_id, chain_id in pdb_info:
    # Construct the RCSB URL for retrieving the FASTA
    url = f"https://www.rcsb.org/fasta/entry/{pdb_id.upper()}?chainId={chain_id.upper()}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError if the request returned an unsuccessful status code
        
        # Define a filename, e.g., "6ksh_D.fasta"
        filename = f"{pdb_id.lower()}_{chain_id.upper()}.fasta"
        
        with open(filename, "w") as outfile:
            outfile.write(response.text)
        
        print(f"Downloaded FASTA for {pdb_id.upper()} chain {chain_id.upper()} into {filename}")
    
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {pdb_id.upper()} chain {chain_id.upper()}: {e}")

Downloaded FASTA for 6KSH chain D into 6ksh_D.fasta
Downloaded FASTA for 4CTA chain A into 4cta_A.fasta
Downloaded FASTA for 2X14 chain A into 2x14_A.fasta
Downloaded FASTA for 2J9C chain A into 2j9c_A.fasta
Downloaded FASTA for 3F5M chain A into 3f5m_A.fasta
Downloaded FASTA for 3RUV chain D into 3ruv_D.fasta
Downloaded FASTA for 5TRD chain A into 5trd_A.fasta
Downloaded FASTA for 3C1M chain C into 3c1m_C.fasta
Downloaded FASTA for 3V2U chain C into 3v2u_C.fasta
Downloaded FASTA for 1F3F chain C into 1f3f_C.fasta
Downloaded FASTA for 7NSD chain A into 7nsd_A.fasta
Downloaded FASTA for 5DD7 chain A into 5dd7_A.fasta
Downloaded FASTA for 1I58 chain A into 1i58_A.fasta
Downloaded FASTA for 3HY2 chain Y into 3hy2_Y.fasta
Downloaded FASTA for 7ULD chain A into 7uld_A.fasta
Downloaded FASTA for 1Z0S chain A into 1z0s_A.fasta
Downloaded FASTA for 1UN9 chain A into 1un9_A.fasta
Downloaded FASTA for 7ALR chain A into 7alr_A.fasta
Downloaded FASTA for 121P chain A into 121p_A.fasta
Downloaded F

In [81]:
import os
import json

# Directory containing the downloaded FASTA files
FASTA_DIR = "fasta"

# This will hold a list of job dictionaries
jobs = []

# Loop over all FASTA files in the directory
for filename in os.listdir(FASTA_DIR):
    if filename.endswith(".fasta"):
        filepath = os.path.join(FASTA_DIR, filename)

        # Example of FASTA filename: "6ksh_D.fasta"
        # Remove the extension -> "6ksh_D"
        base_name = os.path.splitext(filename)[0]  # e.g. "6ksh_D"

        # Split into pdb_id and chain_id if there's an underscore
        if "_" in base_name:
            pdb_id, chain_id = base_name.split("_", maxsplit=1)
        else:
            # If no underscore, assume entire thing is pdb_id (chain = "")
            pdb_id, chain_id = base_name, ""

        # Read lines from FASTA and strip whitespace
        with open(filepath, "r") as f:
            lines = [line.strip() for line in f if line.strip()]

        # Skip empty files
        if not lines:
            continue

        # First line is the header (e.g. ">6KSH_1|Chain D|...")
        header_line = lines[0]
        if header_line.startswith(">"):
            header_line = header_line[1:].strip()

        # The remaining lines are the sequence. Join them into one string.
        sequence = "".join(lines[1:])

        # Construct a job name (you can customize however you like):
        job_name = f"AlphaFold_{pdb_id.upper()}_{chain_id.upper()}" if chain_id else f"AlphaFold_{pdb_id.upper()}"

        # Build the job dictionary
        job_dict = {
            "name": job_name,
            "modelSeeds": [],  # empty = random seed
            "sequences": [
                {
                    "proteinChain": {
                        "sequence": sequence,
                        "count": 1
                    }
                }
            ],
            "dialect": "alphafoldserver",
            # NOTE: Official docs mention up to "version": 1;
            # If your server really supports version 3, set this to 3
            "version": 3
        }

        # Append this job to the list of jobs
        jobs.append(job_dict)

# The final output is a list of job dictionaries
# per the AlphaFold Server JSON file format
output_filename = "alphafold_jobs.json"
with open(output_filename, "w") as outfile:
    json.dump(jobs, outfile, indent=2)

print(f"Created AlphaFold Server JSON (version 3) with {len(jobs)} jobs: {output_filename}")


Created AlphaFold Server JSON (version 3) with 89 jobs: alphafold_jobs.json


In [47]:
pdb_info.index(('6p1p','A'))

61

In [51]:
test = "MGGSHHHHHHRSESTVTEELKEGIDAVYPSLVGTADSKAEGIKNYFKLSFTLPEEQKSRTVGSEAPLKDVAQALSSRARYELFTEKETANPAFNGEVIKRYKELMEHGEGIADILRSRLAKFLNTKDVGKRFAQGTEANRWVGGKLLNIVEQDGDTFKYNEQLLQTAVLAGLQWRLTATSNTAIKDAKDVAAITGIDQALLPEGLVEQFDTGMTLTEAVSSLAQKIESYWGLSRNPNAPLGYTKGIPTAMAAEILAAFVESTDVVENIVDMSEIDPDNKKTIGLYTITELDSFDPINSFPTAIEEAVLVNPTEKMFFGDDIPPVANTQLRNPAVRNTPEQKAALKAEQATEFYVHTPMVQFYETLGKDRILELMGAGTLNKELLNDNHAKSLEGKNRSVEDSYNQLFSVIEQVRAQSEDISTVPIHYAYNMTRVGRMQMLGKYNPQSAKLVREAILPTKATLDLSNQNNEDFSAFQLGLAQALDIKVHTMTREVMSDELTKLLEGNLKPAIDMMVEFNTTGSLPENAVDVLNTALGDRKSFVALMALMEYSRYLVAEDKSAFVTPLYVEADGVTNGPINAMMLMTGGLFTPDWIRNIAKGGLFIGSPNKTMNEHRSTADNNDLYQASTNALMESLGKLRSNYASNMPIQSQIDSLLSLMDLFLPDINLGENGALELKRGIAKNPLTITIYGSGARGIAGKLVSSVTDAIYERMSDVLKARAKDPNISAAMAMFGKQAASEAHAEELLARFLKDMETLTSTVPVKRKGVLELQSTGTGAKGKINPKTYTIKGEQLKALQENMLHFFVEPLRNGITQTVGESLVYSTEQLQKATQIQSVVLEDMFKQRVQEKLAEKAKDPTWKKGDFLTQKELNDIQASLNNLAPMIETGSQTFYIAGSENAEVANQVLATNLDDRMRVPMSIYAPAQAGVAGIPFMTIGTGDGMMMQTLSTMKGAPKNTLKIFDGMNIGLNDITDASRKANEAVYTSWQGNPIKNVYESYAKFMKNVDFSKLSPEALEAIGKSALEYDQRENATVDDIANAASLIERNLRNIALGVDIRHKVLDKVNLSIDQMAAVGAPYQNNGKIDLSNMTPEQQADELNKLFREELEARKQKVAKAR>4FF3_2|Chains B[auth C], D|Bacteriophage N4 P2 promoter|nullTGCCTCCCAGGCATTCAAAAGAAGCGGAGCTTCTTC"

In [59]:
letters = list(test)

In [71]:
allowed_char =['A', 'R', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'N', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']

In [79]:
difference = list(set(letters) - set(allowed_char))